# Ground Truth Entity Recognition

In [1]:
# %pip install awswrangler
# %pip install bleach
# %pip install nltk

In [2]:
import os
import sys
import nltk
import json
import boto3 
import numpy as np
import pandas as pd
import awswrangler as wr
from bleach.sanitizer import Cleaner

nltk.download('punkt')

from sagemaker import get_execution_role
role = get_execution_role()

sys.path.append(os.path.dirname(os.getcwd()) + '/src')
print('added ', os.path.dirname(os.getcwd()) + '/src', ' to sys.')
from preprocess import preprocess_data

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


added  /home/ec2-user/SageMaker/ons-poc-manobras-catalog/src  to sys.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
preproc = preprocess_data()

Preprocessing module
started at  2021-09-30 12:42:16.547523


In [4]:
# reading file contents from s3 bucket
df = preproc.get_baseline_text()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6025 entries, 0 to 6024
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       6025 non-null   int64  
 1   documentUID              6025 non-null   object 
 2   contentUID               6025 non-null   object 
 3   Title                    6023 non-null   object 
 4   Level                    6025 non-null   object 
 5   hierarquia               6025 non-null   object 
 6   relatedEmbedded          0 non-null      float64
 7   relatedLinks             0 non-null      float64
 8   MpoAreaEletrica          6025 non-null   object 
 9   MpoCentro                6025 non-null   object 
 10  MpoEquipamentos          2370 non-null   object 
 11  MpoIdentificador         6025 non-null   object 
 12  MpoLigadoDesligado       6025 non-null   object 
 13  MpoLocalizacaoTaxonomia  6025 non-null   object 
 14  MpoMopsAssociadas       

,Id,documentUID,contentUID,Title,Level,hierarquia,relatedEmbedded,relatedLinks,MpoAreaEletrica,MpoCentro,...,MpoRevisaoReadOnly,MpoSubmodulo,MpoTema,MpoTextoVigenciaPorData,MpoTipoDocumento,nomeDocumento,statusItem,StatusDocumento,texto,Created
0,181505,e401f716-b9c8-44cf-a961-75dd62a2a8aa,cffb9a6b-ba1f-4756-82d4-cd88a5230036,OBJETIVO,1,OBJETIVO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,Estabelecer procedimentos para manobras de equ...,30/07/2021 14:43:17
1,181506,e401f716-b9c8-44cf-a961-75dd62a2a8aa,7b8fcaee-0dce-41f1-b1d9-3dacc9871960,CONSIDERAÇÕES GERAIS,2,CONSIDERAÇÕES GERAIS,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,2.1. Os procedimentos contidos nest...,30/07/2021 14:43:18
2,181507,e401f716-b9c8-44cf-a961-75dd62a2a8aa,5a57338c-7b88-466c-a769-51ce87e128b1,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,3,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,1. OBJETIVO2....................................,30/07/2021 14:43:18
3,181508,e401f716-b9c8-44cf-a961-75dd62a2a8aa,2d387a90-353d-4411-b263-d528fbba8e7c,LT 230 KV Tucuruí / Altamira,3.1,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.1.1. Desenergização da LT 230 kV Tucur...,30/07/2021 14:43:18
4,181509,e401f716-b9c8-44cf-a961-75dd62a2a8aa,0ba555c0-1b6a-4d16-8f76-8758b214042c,LT 230 KV Altamira / Xingu,3.2,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.2.1. Desenergização da LT 230 kV Altam...,30/07/2021 14:43:18


In [7]:
df = df[df['MpoEquipamentos'].isnull()==True]
df.head()

,Id,documentUID,contentUID,Title,Level,hierarquia,relatedEmbedded,relatedLinks,MpoAreaEletrica,MpoCentro,...,MpoRevisaoReadOnly,MpoSubmodulo,MpoTema,MpoTextoVigenciaPorData,MpoTipoDocumento,nomeDocumento,statusItem,StatusDocumento,texto,Created
0,181505,e401f716-b9c8-44cf-a961-75dd62a2a8aa,cffb9a6b-ba1f-4756-82d4-cd88a5230036,OBJETIVO,1,OBJETIVO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,Estabelecer procedimentos para manobras de equ...,30/07/2021 14:43:17
1,181506,e401f716-b9c8-44cf-a961-75dd62a2a8aa,7b8fcaee-0dce-41f1-b1d9-3dacc9871960,CONSIDERAÇÕES GERAIS,2,CONSIDERAÇÕES GERAIS,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,2.1. Os procedimentos contidos nest...,30/07/2021 14:43:18
2,181507,e401f716-b9c8-44cf-a961-75dd62a2a8aa,5a57338c-7b88-466c-a769-51ce87e128b1,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,3,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,1. OBJETIVO2....................................,30/07/2021 14:43:18
9,181514,e401f716-b9c8-44cf-a961-75dd62a2a8aa,6dca753e-cdf4-46b6-a2d9-86709e386b5b,LT(s) 138 KV Rurópolis / Tapajós II com deriva...,3.7,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.7.1. Desenergização da LT 138 KV Rurópo...,30/07/2021 14:43:19
10,181515,e401f716-b9c8-44cf-a961-75dd62a2a8aa,dfa840b5-9625-4d53-a175-f710668090f6,PREPARAÇÃO PARA MANOBRAS DE TRANSFORMADORES,4,PREPARAÇÃO PARA MANOBRAS DE TRANSFORMADORES,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,1. OBJETIVO2....................................,30/07/2021 14:43:19


In [8]:
df['MpoCentro'].value_counts()

COSR-SE     1008
COSR-NE      997
COSR-S       817
COSR-NCO     619
CNOS         214
Name: MpoCentro, dtype: int64

In [9]:
df_list = preproc.separate_df_by_region_with_centerinfo(df, column='MpoCentro')

In [10]:
for sublist in range(len(df_list)):
    print(df_list[sublist][0], '- amostras :', len(df_list[sublist][1]))

CNOS - amostras : 214
COSR-NE - amostras : 997
COSR-NCO - amostras : 619
COSR-SE - amostras : 1008
COSR-S - amostras : 817


In [11]:
bucket = 'ons-dl-dev-landing'
file_prefix = 'ons/mpo_ds_nlp/regioes/'

for sublist in range(len(df_list)):
    
    file_samples = file_prefix + df_list[sublist][0].lower() + '_equipnull' + '/samples.csv'
    #file_index = file_prefix + df_list[sublist][0].lower() + '_equipnull' + '/index.csv'
    
    obj_uri_samples = 's3://{}/{}'.format(bucket, file_samples)
    #obj_uri_index = 's3://{}/{}'.format(bucket, file_index)
    
    df_list[sublist][1]['texto'].to_csv(obj_uri_samples, index=False, header=False)
    #df_list[sublist][1]['Id'].to_csv(obj_uri_index, index=False, header=False)
    print(df_list[sublist][0], ' -> ', obj_uri_samples)

CNOS  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/cnos_equipnull/samples.csv
COSR-NE  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/cosr-ne_equipnull/samples.csv
COSR-NCO  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/cosr-nco_equipnull/samples.csv
COSR-SE  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/cosr-se_equipnull/samples.csv
COSR-S  ->  s3://ons-dl-dev-landing/ons/mpo_ds_nlp/regioes/cosr-s_equipnull/samples.csv



Example of sample identified in NE region:

In [12]:
df_list[0][1]['texto'].iloc[5]

'PassoCoordenaçãoControleComando / ExecuçãoProcedimentosObjetivo / Item de Controle1CNOSCOSR-NE / COSR-NCOPTE (COS-STATE GRID)Baipassar os capacitores série fixos da LT 500 kV Luziânia / Rio das Éguas. 2CNOSCOSR-NCOPTE (COS-STATE GRID)Abrir a LT 500 kV Luziânia / Rio das Éguas no terminal da SE Luziânia.Obs.: O terminal da LT 500 kV Luziânia / Rio das Éguas, na SE Rio das Éguas, será aberto automaticamente por recepção de TDD.         Limitações impostas pela indisponibilidade da LT 500 kV Luziânia / Rio das Éguas atendidas conforme definido na IO-OC.SENE.         SE Luziânia ≤ 542 kV         Capacitores série fixos baipassados. '